In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
import gc
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
!pip install cairocffi
matplotlib.use('Cairo')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D
!pip install -U git+https://github.com/twopirllc/pandas-ta
import pandas_ta as pta


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 657 kB/s 
     |████████████████████████████████| 6.4 MB 11.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████

In [ ]:

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
  if not os.path.exists("/content/checkpoints/"):
    os.mkdir("/content/checkpoints/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)
    data = data.dropna()
    dfs = []
    suggs = []
    df = None
    _a = None
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    #data["6_rsi"] =  pta.rsi(data['Close'], length = 6)
    #data["14_rsi"] = pta.rsi(data['Close'], length = 14)
    #data["26_rsi"] = pta.rsi(data['Close'], length = 26)
    #data["52_rsi"] = pta.rsi(data['Close'], length = 52)

    data = np.array(data)
    data = data.astype(float)
    max_index = data.shape[0]-which_future_or_past
    for i in range(each_row_past,max_index):

        rows = data[i-each_row_past:i, :]
        past_candles = []
        for z in range(1, how_many_past_candles+1):
          past_candles.append((data[i-z][3]+data[i-z][0])/2)
        past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append((data[i+z][3]+data[i+z][0])/2)
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > past_candles:
          sugg = 1
        else:
          sugg = 0

        #x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close","6_rsi", "14_rsi", "26_rsi","52_rsi"])
        df = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close","Adj Close","Volume"])

        df.index.name = "Date"
        #date_list = []
        #rng = np.array(df).shape[0]
        #for i in range(rrng):
        #    date_list.append(datetime.fromordinal(
        #        datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        #df.index = date_list
        #df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{sugg}.png"
        
        #apds = [ 
        # mpl.make_addplot(df["6_rsi"],color='b',width=2),
        # mpl.make_addplot(df["14_rsi"],color='g',width=2),
        # mpl.make_addplot(df["26_rsi"],color='r',width=2),
        # mpl.make_addplot(df["52_rsi"],color='y',width=2),
       #]
        
        fig, _a = mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                            returnfig=True, tight_layout=True,figsize =(1.5,1.5))
        
        fig.savefig(address)
        fig.clear()
    
        if i % 10 == 0:
            print(f"{i}/{max_index}", end=" ")
        if i % 160 == 0:
          print(" ")
        if i % 19 == 0:
          plt.close("all")
    plt.close("all")

    print("")


def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [ ]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["btc-usd","eth-usd","trx-usd","ltc-usd","xrp-usd","bnb-usd"]
download_data(symbols,"max","1d")

In [ ]:
how_many_future_candles = 20
how_many_past_candles = 1
each_row_past = 150

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 6
Processing File:
File Number 1: 10/1548 20/1548 30/1548 40/1548 50/1548 60/1548 70/1548 80/1548 90/1548 100/1548 110/1548 120/1548 130/1548 140/1548 150/1548 160/1548  
170/1548 180/1548 190/1548 200/1548 210/1548 220/1548 230/1548 240/1548 250/1548 260/1548 270/1548 280/1548 290/1548 300/1548 310/1548 320/1548  
330/1548 340/1548 350/1548 360/1548 370/1548 380/1548 390/1548 400/1548 410/1548 420/1548 430/1548 440/1548 450/1548 460/1548 470/1548 480/1548  
490/1548 500/1548 510/1548 520/1548 530/1548 540/1548 550/1548 560/1548 570/1548 580/1548 590/1548 600/1548 610/1548 620/1548 630/1548 640/1548  
650/1548 660/1548 670/1548 680/1548 690/1548 700/1548 710/1548 720/1548 730/1548 740/1548 750/1548 760/1548 770/1548 780/1548 790/1548 800/1548  
810/1548 820/1548 830/1548 840/1548 850/1548 860/1548 870/1548 880/1548 890/1548 900/1548 910/1548 920/1548 930/1548 940/1548 950/1548 960/1548  
970/1548 980/1548 990/1548 1000/1548 1010/1548 1020/1548 1030/1548 1040/1548 1050/1

11586

In [ ]:
shutil.make_archive(folder_name,"zip",f"/content/extracted/{folder_name}/")
#shutil.unpack_archive("/content/extracted/084505.zip","/content/extracted/455")

'/content/120943.zip'

In [ ]:
#folder_name = '112904'

label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name)

    dim = (120, 90)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
data = np.array(data)
data = data / 255
print(data.shape)
xTrain , xTest , yTrain , yTest = train_test_split(data,label,test_size=0.2,random_state=99)

data = None
label = None
print(f"xTrain : {len(xTrain)} \\ xTest : {len(xTest)}")

nytrain = []
nytest = []
yn = 0
nn = 0

for i in yTrain:
  if i == 1:
    nytrain.append([1,0])
    yn += 1
  else:
    nytrain.append([0,1])
    nn += 1
for i in yTest:
  if i == 1:
    nytest.append([1,0])
    yn += 1
  else:
    nytest.append([0,1])
    nn += 1

yTrain = np.array(nytrain)
yTest = np.array(nytest)

print(f"yn: {yn} nn: {nn}")

(11586, 90, 120, 3)
xTrain : 9268 \ xTest : 2318
yn: 6178 nn: 5408


In [ ]:
model = Sequential()

model.add(Conv2D(128,    (3, 3),activation="relu", input_shape=(xTrain.shape[1], xTrain.shape[2],3)))
model.add(Conv2D(100,     (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,    (3, 3),activation="relu",))
model.add(Conv2D(32,    (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,    (3, 3),activation="relu",))
model.add(Conv2D(32,    (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(750,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adamax,
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 88, 118, 128)      3584      
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 117, 100)      51300     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 43, 58, 100)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 56, 64)        57664     
                                                                 
 conv2d_3 (Conv2D)           (None, 40, 55, 32)        8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 27, 32)       0         
 2D)                                                    

In [ ]:
filepath = "/content/checkpoints/{epoch:02d}-{val_accuracy:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#model.fit(xTrain,yTrain,batch_size=64,epochs=30,validation_data=(xTest,yTest), callbacks=model_checkpoint_callback)
model.fit(xTrain,yTrain,batch_size=8,epochs=20,validation_data=(xTest,yTest))


Epoch 1/20
1159/1159 [==============================] - 1225s 1s/step - loss: 0.6921 - accuracy: 0.5296 - val_loss: 0.6916 - val_accuracy: 0.5393
Epoch 2/20
1159/1159 [==============================] - 1230s 1s/step - loss: 0.6915 - accuracy: 0.5306 - val_loss: 0.6903 - val_accuracy: 0.5393
Epoch 3/20
1159/1159 [==============================] - ETA: 0s - loss: 0.6915 - accuracy: 0.5317